In [124]:
import re
import pandas as pd
import numpy as np

In [125]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Twitter/"
readme = open(f"{main_path}Input/README.txt", "r")
readme =readme.read()
print(readme)
type(readme)

INTRODUCTION
This archive was generated at the request of the following user:
- @username at the time the archive was generated: daggeo91
- Account ID: 1854507704154497024

The easiest way to navigate your archive is to open the HTML renderer in a desktop web browser by double clicking the “Your archive” file included in the main folder once the archive is unzipped.

Note that the HTML renderer only works if the archive is less than 50GB. Also note that the HTML renderer only includes a subset of the data included in the archive. To see all the data included in the archive, please navigate the JSON files located in the “data” folder.

The data folder consists of machine-readable JSON files with a .js extension containing information associated with this account. We’ve included the information we believe is most relevant and useful, including profile information, Tweets, Direct Messages, Moments, images, videos and GIFs attached to Tweets, Direct Messages or Moments, followers, followin

str

In [126]:
readme = re.split("----------------------", readme)


In [127]:
readme_df= pd.DataFrame(readme, columns = ['full_strings'] )

In [128]:
pattern = r'\b[\w\-]+\.js\b'
readme_df['file_folder_name'] = readme_df['full_strings'].str.extract(f'({pattern})')


In [129]:
readme_df['type'] = readme_df.apply(
    lambda row: 'js_file' if pd.notna(row['file_folder_name']) else 'folder', axis=1)

In [130]:
#pattern = r'^\s*-\s*[^:]+:\s*(.+)$'
pattern = pattern = r'-\s([\w\-]+)\:\s*(.+)'

readme_df['description'] = readme_df['full_strings'].str.findall(pattern, flags=re.MULTILINE)


In [131]:
readme_df = readme_df.explode('description')

In [132]:
readme_df.dtypes['description']

dtype('O')

In [133]:


readme_df['description']  = readme_df['description'] .apply(lambda x: str(x))

In [134]:
readme_df['description'] = readme_df['description'].str.replace('(', '')
readme_df['description'] = readme_df['description'].str.replace(')', '')

In [135]:
# new data frame with split value columns
new = readme_df['description'].str.split(',', n=1, expand=True)

# making separate first name column from new data frame
readme_df['key'] = new[0]

# making separate last name column from new data frame
readme_df['description'] = new[1]

readme_df['description'] = readme_df['description'].str.replace("'", '')
readme_df['key'] = readme_df['key'].str.replace("'", '')


In [136]:
readme_df['type'] = readme_df.apply(
    lambda row: 'js_file' if pd.notna(row['file_folder_name']) else 'folder', axis=1)

In [137]:
readme_df.loc[readme_df['type'] == 'folder', 'file_folder_name'] = \
    readme_df.loc[readme_df['type'] == 'folder', 'full_strings'].apply(lambda x: str(x).split(' ')[0])


In [138]:
readme_df.loc[readme_df['type'] == 'folder', 'description'] = \
    readme_df.loc[readme_df['type'] == 'folder', 'full_strings'].apply(lambda x: ' '.join(str(x).split(' ')[1:]))


In [139]:
readme_df = readme_df[['file_folder_name', 'type','key', 'description', 'full_strings']]

In [140]:
readme_df.to_csv(f"{main_path}Final/Merged_structures_X.csv", index=False)